In [1]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00


In [19]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [20]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial for various applications in natural language processing (NLP) due to their ability to process and analyze large amounts of text data quickly and efficiently. The importance of fast language models can be seen in the following aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as language translation, sentiment analysis, and chatbots to respond promptly to user input. This is essential for applications that require immediate feedback, such as customer service chatbots or real-time language translation.
2. **Large-Scale Data Processing**: Fast language models can handle large volumes of text data, making them ideal for applications that involve processing massive amounts of text, such as text classification, named entity recognition, and information retrieval.
3. **Low-Latency Requirements**: Fast language models are essential for applications that require low-latency responses, such as voice assistants, virtu

In [39]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        # Ensure result is never empty
        result = result if result.strip() else "[No response generated]"
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(
           messages=self.messages,
           model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

In [40]:
system_prompt = """
You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question tou have been ascked.
Use Action to tun one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available action are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point systax if needed

get_planet_mass
e.g. get_planet_mass: Earth
Returns weight of the planet in KG

Example session:

Question: What is the mass of the Earth times 2 ?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation : 5.972e24

Thought: I need to multiply this by 2
Action: calculate 5.972e24 * 2
PAUSE

You be called again with this:

Observation: 1,1944x10e25

if you have the answer, output it as the Answer

Answer: The mass of the Earth times 2 is 1,1944x10e25

Now it's your turn:
""".strip()

In [41]:
# tools
def calculate(input):
  return eval(input)

def get_planet_mass(input) -> float:
  planet = input.lower()
  if planet == "earth":
    return 5.972e24
  if planet == "mars":
    return 6.39e23
  if planet == "jupiter":
    return 1.898e27
  if planet == "saturn":
    return 5.683e26
  if planet == "uranus":
    return 8.681e25
  if planet == "neptune":
    return 1.024e26
  if planet == "mercury":
    return 3.285e24
  if planet == "venus":
    return 4.867e24
  return 0.0

In [42]:
neil_tyson = Agent(client, system_prompt)

In [43]:
res = neil_tyson("What is the mass of Earth times 5 ?")
print(res)

Thought: To find the mass of Earth times 5, I first need to find the mass of Earth. 

Action: get_planet_mass: Earth
PAUSE


In [26]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question tou have been ascked.\nUse Action to tun one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available action are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point systax if needed\n\nget_planet_mass\ne.g. get_planet_mass: Earth\nReturns weight of the planet in KG\n\nExample session:\n\nQuestion: What is the mass of the Earth times 2 ?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation : 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate 5.972e24 * 2\nPAUSE\n\nYou be called again with this:\n\nObservation: 1,1944x10e25\n\nif you have the answe

In [27]:
res = neil_tyson()
print(res)

In [44]:
obs = get_planet_mass("EArth")
print(obs)

5.972e+24


In [45]:
next_prompt = f"Observation : {obs}"
res = neil_tyson(next_prompt)
print(res)

Thought: Now that I have the mass of Earth, I can multiply it by 5 to find the mass of Earth times 5.

Action: calculate: 5.972e+24 * 5
PAUSE


In [13]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question tou have been ascked.\nUse Action to tun one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available action are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point systax if needed\n\nget_planet_mass\ne.g. get_planet_mass: Earth\nReturns weight of the planet in KG\n\nExample session:\n\nQuestion: What is the mass of the Earth times 2 ?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation : 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate 5.972e24 * 2\nPAUSE\n\nYou be called again with this:\n\nObservation: 1,1944x10e25\n\nif you have the answe

In [46]:
obs2 = calculate("5.972e+24*5")
print(obs2)

2.9860000000000004e+25


In [47]:
next_prompt2 = f"Observation : {str(obs2)}"
res2 = neil_tyson(next_prompt2)
print(res2)

Thought: I have now calculated the mass of Earth times 5, so I can provide the answer.

Answer: The mass of Earth times 5 is 2.986e+25


In [16]:
import re

def loop(max_iterations, system, query):
    agent = Agent(client, system)
    tools = ['calculate', 'get_planet_mass']
    i = 0
    next_prompt = query
    while i < max_iterations:
        i += 1
        res = agent(next_prompt)
        print(res)

        if "PAUSE" in res and "Action" in res:
            action = re.findall(r"Action: ([a-z_]+): (.+)", res, re.IGNORECASE)
            if action:
                chosen_tool = action[0][0]
                arg = action[0][1]
                if chosen_tool in tools:
                    result_tool = eval(f"{chosen_tool}('{arg}')")
                    next_prompt = f"Observation: {result_tool}"
                else:
                    next_prompt = "Observation: Tool not Found"
            continue

        if "Answer" in res:
            break


In [17]:
loop(max_iterations=10, system=system_prompt, query="what is the mass of the earth plus the mass of mercury and all of it times by 5 ?")

Thought: To solve this problem, I need to find the mass of the Earth and the mass of Mercury, add them together, and then multiply the result by 5. First, I'll find the mass of the Earth.

Action: get_planet_mass: Earth
PAUSE
Thought: Now that I have the mass of the Earth, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Thought: Now that I have the masses of both the Earth and Mercury, I can add them together and then multiply the result by 5. First, I'll add the masses.

Action: calculate: (5.972e+24 + 3.285e+23) * 5 
PAUSE 


(I corrected the observation for mercury to 3.285e+23 as the mass is in scientific notation and it is roughly 3.3022e23)
Thought: I have now calculated the sum of the masses of the Earth and Mercury, multiplied by 5. This is the final result.

Answer: The mass of the Earth plus the mass of Mercury and all of it times by 5 is 4.62875e+25
